In [2]:
from bs4 import BeautifulSoup
import requests
import sqlite3
import pandas as pd
import time
from random import random


TIKI_URL = 'https://tiki.vn'

In [4]:
conn = sqlite3.connect('tiki.db')
cur = conn.cursor()

In [5]:
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}


In [6]:
# Get the HTML content get_url()
def get_url(url):
    try:
        response = requests.get(url,headers = HEADERS).text
        soup = BeautifulSoup(response, 'html.parser')
        return soup
    except Exception as err:
        print('ERROR BY REQUEST:', err)
        

In [7]:
CATEGORY_SET = set()
def can_add_to_cat_set(cat_name,save=False):
  if cat_name not in CATEGORY_SET:
    if save:
      CATEGORY_SET.add(cat_name)
      print(f'Added "{cat_name}" to CATEGORY_SET')
    return True
  return False

In [8]:
def get_main_categories(save_db=False):
    soup = get_url(TIKI_URL)

    result = []
    for a in soup.find_all('a', {'class': 'menu-link'}):
        name = a.find('span', {'class': 'text'}).text.strip()
        
        _=can_add_to_cat_set(name,save_db)

        url = a['href']
        main_cat = Category(name, url) # object from class Category

        if save_db:
            main_cat.save_into_db()
        result.append(main_cat)
    return result

In [9]:
import re

# get_sub_categories() given a parent category
def get_sub_categories(parent_category, save_db=False):
    parent_url = parent_category.url
    result = []

    try:
        soup = get_url(parent_url)
        for a in soup.find_all('a', {'class':'item item--category '}):
            name = a.text.strip()
            if can_add_to_cat_set(name,save_db): 
              sub_url = a['href']
              cat = Category(name, sub_url, parent_category.cat_id) # we now have parent_id, which is cat_id of parent category
              if save_db:
                  cat.save_into_db()
              result.append(cat)
    except Exception as err:
        print('ERROR IN GETTING SUB CATEGORIES:', err)
    return result

In [10]:
pd.read_sql_query('SELECT * FROM categories', conn).head(10)

,id,name,url,parent_id,create_at
0,1,Điện Thoại - Máy Tính Bảng,https://tiki.vn/dien-thoai-may-tinh-bang/c1789...,NaN,2020-11-28 10:43:37
1,2,Điện Tử - Điện Lạnh,https://tiki.vn/tivi-thiet-bi-nghe-nhin/c4221?...,NaN,2020-11-28 10:43:37
2,3,Phụ Kiện - Thiết Bị Số,https://tiki.vn/thiet-bi-kts-phu-kien-so/c1815...,NaN,2020-11-28 10:43:37
3,4,Laptop - Thiết bị IT,https://tiki.vn/laptop-may-vi-tinh/c1846?src=c...,NaN,2020-11-28 10:43:37
4,5,Máy Ảnh - Quay Phim,https://tiki.vn/may-anh/c1801?src=c.1801.hambu...,NaN,2020-11-28 10:43:37
5,6,Điện Gia Dụng,https://tiki.vn/dien-gia-dung/c1882?src=c.1882...,NaN,2020-11-28 10:43:37
6,7,Nhà Cửa Đời Sống,https://tiki.vn/nha-cua-doi-song/c1883?src=c.1...,NaN,2020-11-28 10:43:37
7,8,Hàng Tiêu Dùng - Thực Phẩm,https://tiki.vn/bach-hoa-online/c4384?src=c.43...,NaN,2020-11-28 10:43:37
8,9,"Đồ chơi, Mẹ & Bé",https://tiki.vn/me-va-be/c2549?src=c.2549.hamb...,NaN,2020-11-28 10:43:37
9,10,Làm Đẹp - Sức Khỏe,https://tiki.vn/lam-dep-suc-khoe/c1520?src=c.1...,NaN,2020-11-28 10:43:37


In [11]:
pd.read_sql_query('SELECT COUNT(Name) FROM categories', conn)

,COUNT(Name)
0,2595


In [38]:
query = pd.read_sql_query('''
WITH tmp AS(
      SELECT a.name, a.url, a.id, a.parent_id, b.id
      FROM categories AS a
      LEFT JOIN categories AS b
        ON a.id = b.parent_id
      WHERE b.id IS NULL
      )
SELECT * 
FROM tmp
WHERE parent_id IS NOT NULL
''', conn)

In [39]:
query

,name,url,id,parent_id,id:1
0,Máy tính bảng,https://tiki.vn/may-tinh-bang/c1794?src=c.1789...,17,1,None
1,Máy đọc sách,https://tiki.vn/may-doc-sach/c28856?src=c.1789...,18,1,None
2,Điện thoại Smartphone,https://tiki.vn/dien-thoai-smartphone/c1795?sr...,19,1,None
3,Điện thoại bàn,https://tiki.vn/dien-thoai-ban/c8061?src=c.178...,20,1,None
4,Điện thoại phổ thông,https://tiki.vn/dien-thoai-pho-thong/c1796?src...,21,1,None
...,...,...,...,...,...
2143,Spa - Thẩm mỹ viện,https://tiki.vn/spa-tham-my-vien/c11323?src=c....,2591,2551,None
2144,GYM,https://tiki.vn/gym/c13366?src=c.11312.hamburg...,2592,2588,None
2145,Ca nhạc - Phim - Kịch,https://tiki.vn/ca-nhac-phim-kich/c11328?src=c...,2593,2552,None
2146,Studio - Chụp ảnh,https://tiki.vn/studio-chup-anh/c11330?src=c.1...,2594,2552,None


In [40]:
query.head()

,name,url,id,parent_id,id:1
0,Máy tính bảng,https://tiki.vn/may-tinh-bang/c1794?src=c.1789...,17,1,None
1,Máy đọc sách,https://tiki.vn/may-doc-sach/c28856?src=c.1789...,18,1,None
2,Điện thoại Smartphone,https://tiki.vn/dien-thoai-smartphone/c1795?sr...,19,1,None
3,Điện thoại bàn,https://tiki.vn/dien-thoai-ban/c8061?src=c.178...,20,1,None
4,Điện thoại phổ thông,https://tiki.vn/dien-thoai-pho-thong/c1796?src...,21,1,None


In [41]:
parent = query["parent_id"]
url = query["url"]

In [42]:
def tiki_scraper(category_url, parent_id,data):
    page=1
    while page < 3:
        # generates random delay between 0.5 and 1.5 seconds:
        delay = random() + 0.5

        # defines target page url 
        current = category_url + str(page)

        # general url business and creating soup obj:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

        r = requests.get(current, headers=headers)
        soup = BeautifulSoup(r.text, "html.parser")

        # populates list of all products on target page:
        products = soup.find_all("a", {"class": "product-item"}) 
        if len(products)==0:
            products = soup.find_all("div", {"class": "product-item"})
       
        for product in products:
            #creates dictionary of product info:
            d = {}
            d['product_id'] = (product['href'].split('-')[-1])[:-5]    
            d['title'] = product.find('div', {'class':'name'}).text
            price = product.find('div', {'class':'price-discount__price'}).text
            d["price"] = int(price.replace(".","").strip("₫").strip())
            d['image_url'] = product.img['src']
            d["parent_id"] = parent_id
            d['free_shipping'] = bool(product.find('div', {'class':'badge-top'}))
            print(d)
            data.append(d)
        page += 1

        #delays next loop initiation
        time.sleep(delay)

In [61]:
data = []
for i in range(len(url)):
  tmp_url = url[i]
  current_id = parent[i]
  tiki_scraper(tmp_url, current_id, data)

{'product_id': 'p67707109', 'title': 'iPad 10.2 Inch WiFi 32GB (Gen 8) New 2020 - Hàng  Chính Hãng', 'price': 8490000, 'image_url': 'https://salt.tikicdn.com/ts/upload/f3/74/46/f4c52053d220e94a047410420eaf9faf.png', 'parent_id': 1, 'free_shipping': True}
{'product_id': 'p36651420', 'title': 'Máy Tính Bảng Samsung Galaxy Tab S6 (6GB/128GB) - Hàng chính hãng', 'price': 13490000, 'image_url': 'https://salt.tikicdn.com/ts/upload/f3/74/46/f4c52053d220e94a047410420eaf9faf.png', 'parent_id': 1, 'free_shipping': True}
{'product_id': 'p51428406', 'title': 'iPad Pro 11 inch (2020) Wifi - Hàng Nhập Khẩu Chính Hãng', 'price': 20490000, 'image_url': 'https://salt.tikicdn.com/ts/upload/f3/74/46/f4c52053d220e94a047410420eaf9faf.png', 'parent_id': 1, 'free_shipping': True}
{'product_id': 'p67707100', 'title': 'iPad 10.2 Inch WiFi 32GB (Gen 8) New 2020 - Hàng Nhập Khẩu Chính Hãng', 'price': 8750000, 'image_url': 'https://salt.tikicdn.com/ts/upload/f3/74/46/f4c52053d220e94a047410420eaf9faf.png', 'parent

KeyboardInterrupt: 

In [55]:
def create_products_table():
    query = """
        CREATE TABLE IF NOT EXISTS products (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title VARCHAR(255),
            product_id TEXT,
            parent_id INT64,
            price INT64,
            image_url TEXT,
            free_shipping SMALL INT, 
            create_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP

        )
    """
    try:
        cur.execute(query)
        conn.commit()
    except Exception as err:
        print('ERROR BY CREATE TABLE', err)

In [60]:
create_products_table()

In [50]:
type(data[0]["parent_id"])

numpy.int64

In [63]:
for i in data:
  query = """
    INSERT INTO products (title, product_id, parent_id, price, image_url, free_shipping)
    VALUES (?, ?, ?, ?, ?, ?);
"""
  val = (i["title"], i["product_id"], int(i["parent_id"]), i["price"], i["image_url"], i["free_shipping"])

  try:
      cur.execute(query, val)
      cat_id = cur.lastrowid
      conn.commit()

  except Exception as err:
      print('ERROR BY INSERT:', err)

In [59]:

cur.execute('DROP TABLE products;')
conn.commit()

# re-create our category table again


In [64]:
pd.read_sql_query('SELECT * FROM products', conn)

,id,title,product_id,parent_id,price,image_url,free_shipping,create_at
0,1,iPad 10.2 Inch WiFi 32GB (Gen 8) New 2020 - Hà...,p67707109,1,8490000,https://salt.tikicdn.com/ts/upload/f3/74/46/f4...,1,2020-11-29 17:13:14
1,2,Máy Tính Bảng Samsung Galaxy Tab S6 (6GB/128GB...,p36651420,1,13490000,https://salt.tikicdn.com/ts/upload/f3/74/46/f4...,1,2020-11-29 17:13:14
2,3,iPad Pro 11 inch (2020) Wifi - Hàng Nhập Khẩu ...,p51428406,1,20490000,https://salt.tikicdn.com/ts/upload/f3/74/46/f4...,1,2020-11-29 17:13:14
3,4,iPad 10.2 Inch WiFi 32GB (Gen 8) New 2020 - Hà...,p67707100,1,8750000,https://salt.tikicdn.com/ts/upload/f3/74/46/f4...,1,2020-11-29 17:13:14
4,5,iPad Mini 5 Wi-Fi 64GB - Hàng Nhập Khẩu Chính...,p12389305,1,8990000,https://salt.tikicdn.com/ts/upload/f3/74/46/f4...,1,2020-11-29 17:13:14
...,...,...,...,...,...,...,...,...
13887,13888,"Micro Livestream Cho Game Thủ, Giáo Viên Giảng...",p58137503,219,3555555,https://salt.tikicdn.com/ts/upload/f3/74/46/f4...,1,2020-11-29 17:13:21
13888,13889,Mic để bàn dnah cho hội nghị M-3,p71101533,219,179000,https://salt.tikicdn.com/ts/upload/f3/74/46/f4...,1,2020-11-29 17:13:21
13889,13890,Micro Audio Technica ATR2500X-USB - USB Conde...,p68032251,219,3190000,https://salt.tikicdn.com/ts/upload/f3/74/46/f4...,1,2020-11-29 17:13:21
13890,13891,Mic để bàn dành cho hội nghị cao cấp M-13,p71102002,219,369000,https://salt.tikicdn.com/ts/upload/f3/74/46/f4...,1,2020-11-29 17:13:21
